# Import Libraries

In [1]:
import os,sys,re,glob,ipykernel,tweepy,stockmarket,nltk,collections,itertools,pandas as pd,numpy as np,\
        seaborn as sns, yfinance as yf, matplotlib.pyplot as plt, statsmodels.formula.api as smf,\
        statsmodels.api as sm, autoreload, importlib
from pathlib import Path
from string import punctuation 
from datetime import date
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
np.random.seed(0)
pd.set_option('display.max_columns', None)

## Set Working Directory: 
    
* /Sentiment_Analysis 
    
* __ file __ isn't available in jupyter notebooks

    

In [2]:
file = os.getcwd().split(os.sep)
while(file[-1] != 'Sentiment_Analysis'): # Check the working directory
    os.chdir('..')
    file = os.getcwd().split(os.sep)
    sys.path.append(os.path.abspath(os.getcwd()))
print(f"root directory: {os.getcwd()}", sep = '\n')

root directory: c:\Code\Public_Github\Sentiment_Analysis


## Load Custom Functions

In [3]:
from src import user_download_helper, user_download, merge_files, merge_all, \
                strip_all_words, sentence_word_probability, download_todays_test, \
                format_model,linear_model, naive_bayes, create_target, normalize_columns, normalize_columns_target

# Twitter API Credentials

In [4]:
# Read in keys from a csv file
autentication_path = os.path.abspath('../Sentiment_Analysis/Stock_Market/authentication/authentication_tokens.csv')
readin_authentication = pd.read_csv(autentication_path, header=0, sep=',')

consumer_key = readin_authentication['consumer_key'][0]
consumer_secret = readin_authentication['consumer_secret'][0]
access_token = readin_authentication['access_token'][0]
access_token_secret = readin_authentication['access_token_secret'][0]
bearer_token = readin_authentication['beaker_token'][0]

# connect to twitter application 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
redirect_url = auth.get_authorization_url()
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit = True)

# Load Twitter Usernames
* Unvarified user's are not a problem, no one user can have the same ID
    
| Removed User's | reason | 
| ------------ | ------------- |
|DayTradeWarrior|account removed from site|
|elonmusk|privated account|

In [5]:
with open(os.path.normpath(os.getcwd() + '/Stock_Market/user_list/user_list.xlsx'), 'rb') as f:
    user_df = pd.read_excel(f, sheet_name='user_names')
    user_df = user_df.where(pd.notnull(user_df), '')
    f.close()
groups = list(user_df.columns)
user_df

,short_term,long_term,controversial
0,DanZanger,jimcramer,JeffBezos
1,prrobbins,KennethLFisher,BillGates
2,markminervini,lei_zhang_lz,
3,bsc_daily,realwillmeade,
4,MITickWatcher,RayDalio,
5,OptionsProVol,GRDecter,
6,script_crypto,andrewrsorkin,
7,MarketMagnifier,EconguyRosie,
8,TwentyonTwenty_,AswathDamodaran,
9,WatcherGuru,cstewartcfa,


## Download Tweets

### WARNING ~ 4 minutes
    If previously loaded SKIP to CHECKPOINT 
    * Download User tweets into csv spreadsheets 

- ( Tweepy limit of 3600 tweets per user )
    

In [6]:
for group in groups:
    print(f"\n{group}:\n")
    users = list(user_df[group][user_df[group]!= ''])
    user_download(api, users, group)
    print(f"")


short_term:

DanZanger prrobbins markminervini bsc_daily MITickWatcher OptionsProVol script_crypto MarketMagnifier TwentyonTwenty_ WatcherGuru DipFinding MacroCharts techbudsolution eWhispers HindenburgRes JehoshaphatRsch ResearchGrizzly biancoresearch muddywatersre 

long_term:

jimcramer KennethLFisher lei_zhang_lz realwillmeade RayDalio GRDecter andrewrsorkin EconguyRosie AswathDamodaran cstewartcfa BobPisani 

controversial:

JeffBezos BillGates 


## Merge Tweets

In [7]:
merge = []
for group in groups:
    merge.append(merge_files(group, display = 0))
df_all = merge_all('merge/merged_twitter_users', display = 0)

size of merged data sets of short_term: (46007, 7)
size of merged data sets of long_term: (29355, 7)
size of merged data sets of controversial: (3389, 7)
size of merged data sets of merged_twitter_users: (78751, 7)


In [8]:
df_all.head(2)

,id,created_at,user,favorite_count,retweet_count,url,text
0,1621585019742420994,2023-02-03 14:02:50-05:00,WatcherGuru_twitter,157,24,https://twitter.com/i/web/status/1621585019742...,NaN
1,1621583216237756423,2023-02-03 13:55:40-05:00,WatcherGuru_twitter,1886,371,https://twitter.com/i/web/status/1621583216237...,JUST IN Australia releases plans to regulate B...


In [9]:
display(df_all.info(verbose = True, null_counts = None, show_counts=None))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78751 entries, 0 to 3388
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              78751 non-null  int64 
 1   created_at      78751 non-null  object
 2   user            78751 non-null  object
 3   favorite_count  78751 non-null  int64 
 4   retweet_count   78751 non-null  int64 
 5   url             78751 non-null  object
 6   text            76818 non-null  object
dtypes: int64(3), object(4)
memory usage: 4.8+ MB


None

- Some users have infrequent tweets and span the 3600 limit over 10 years

In [10]:
df_all.groupby('user')['created_at'].min().sort_values(ascending= True).head(5)

user
AswathDamodaran_twitter    2010-01-24 19:36:43-05:00
cstewartcfa_twitter        2013-06-19 16:24:32-04:00
BillGates_twitter          2013-07-03 13:04:11-04:00
BobPisani_twitter          2015-11-12 12:26:55-05:00
JeffBezos_twitter          2015-11-24 06:14:26-05:00
Name: created_at, dtype: object

## Drop Old Tweets
- Keep 2017 - 2023

In [11]:
threshold = '2017-01-01'
df_all_upperbound = df_all[df_all.created_at > threshold]
df_all_upperbound.tail(5)

,id,created_at,user,favorite_count,retweet_count,url,text
2144,819411084817924096,2017-01-12 00:10:01-05:00,BillGates_twitter,5351,1724,https://twitter.com/i/web/status/8194110848179...,Here are five things that make me more optimis...
2145,818672177537028096,2017-01-09 23:13:51-05:00,BillGates_twitter,5983,2187,https://twitter.com/i/web/status/8186721775370...,Theres a lot to be optimistic about in 2017lif...
2146,818130649626382336,2017-01-08 11:22:01-05:00,BillGates_twitter,4953,1210,https://twitter.com/i/web/status/8181306496263...,I had a first in 2016I sniffed poop perfume Tr...
2147,817743090526121984,2017-01-07 09:42:00-05:00,BillGates_twitter,7296,2227,https://twitter.com/i/web/status/8177430905261...,I got to learn about this fascinating HIVpreve...
2148,817001964487774208,2017-01-05 08:37:02-05:00,BillGates_twitter,8285,2243,https://twitter.com/i/web/status/8170019644877...,When I was in my 20s and early 30s I was fanat...


In [12]:
# Adding nonessential twitter words to remove
stop = nltk.corpus.stopwords.words("english") 
twitter_nonessential_words = ['twitter', 'birds','lists','list', 'source','just','am','pm'\
                              'a','b','c','d','e','f','g','h','i','j','k','l','m','n',\
                              'n','o','p','q','r','s','t','u','v','w','x','y','z']
stop.extend(twitter_nonessential_words) # merge two lists together
stop = sorted(list( dict.fromkeys(stop) )) # remove duplicates

### Create dictionarys of words 
* Remove unnecessary words
* Generate frequency of words per sentence

In [13]:
df_all_words = strip_all_words(df_all_upperbound, stop)
df_all_words_count = df_all_words.explode().replace("", np.nan, regex=True).dropna() # drop NAN's and empty words
all_count = df_all_words_count.value_counts()

In [14]:
print(f"Tweets of Dictionaries: {len(df_all_words)}")
print(f"all words: {len(df_all_words_count)}")
print(f"Dictionary of all words: {len(all_count)}")

Tweets of Dictionaries: 75766
all words: 1079622
Dictionary of all words: 46700


* Nan are tweets w/ images
* ',' are words removed with special cases

In [15]:
print(f"All the words in each individual Sentence:\n{df_all_words[0:5]}")

All the words in each individual Sentence:
0                                                  NaN
1    [, australia, releases, plans, regulate, bitco...
2                                                  NaN
3                                                  NaN
4                                                  NaN
Name: text, dtype: object


In [16]:
print(f"5 words from dictionary of all words:\n{all_count[0:5]}", end='\n\n')

5 words from dictionary of all words:
stocks    9289
stock     9124
today     6183
score     5866
market    5839
Name: text, dtype: int64



In [17]:
print(all_count.isna().value_counts())

False    46700
Name: text, dtype: int64


# Probability small example

p = count(particular word in sentence) / (total particular word in all sentences) * 100 / (total of all unique words)

d{  hat:1, sandwich:2, lemon:1, orange:1, snorkle:1 }

n = LEN(d.KEYS())  -> n = 5

Tweet1: hat sandwich lemon 

Tweet2: snorkle sandwich orange 

Tweet1:

-> 1/1 * 100 + 1/2 * 100 + 1/1 * 100   
-> 100 + 50 + 100 = 250
-> 250/5 = 50%

Tweet2:

-> 1/1 * 100 + 1/2 * 100 + 1/1 * 100 
-> 100 + 50 + 100 
-> 250/5 = 50%

## Probability of individual tweets

In [18]:
# Probabilities
sentence_list, total_probability, individual_probability = sentence_word_probability(all_count, df_all_words)
print(f'sum of probability column = {sum(total_probability)}')

sum of probability column = 99.99999999999827


In [19]:
df_all_prob = df_all_upperbound.reset_index()
df_all_prob['frequency'] = sentence_list
df_all_prob['probability'] = total_probability
df_all_prob = df_all_prob.dropna()
df_all_prob.insert(loc = 0, column = 'date', value = pd.to_datetime(df_all_prob['created_at']).apply(lambda x: x.strftime('%Y-%m-%d')))
df_all_prob.date = pd.to_datetime(df_all_prob['date'], format='%Y-%m-%d')
df_all_prob = df_all_prob.sort_values(by=['date'], ascending=False).drop(columns=['index'])

In [20]:
df_all_prob.head(2)

,date,id,created_at,user,favorite_count,retweet_count,url,text,frequency,probability
1,2023-02-03,1621583216237756423,2023-02-03 13:55:40-05:00,WatcherGuru_twitter,1886,371,https://twitter.com/i/web/status/1621583216237...,JUST IN Australia releases plans to regulate B...,"[{'australia': 4.166666666666666, 'releases': ...",0.000242
34311,2023-02-03,1621529855392186370,2023-02-03 10:23:38-05:00,DipFinding_twitter,0,0,https://twitter.com/i/web/status/1621529855392...,LIAN down 1048 at 20501 Volume 118k AvgVolume ...,"[{'lian': 14.285714285714285, 'volume': 0.0297...",0.000308


In [21]:
df_wide1 = df_all_prob.pivot_table(index='date', values=['favorite_count','retweet_count'], aggfunc='sum',fill_value=0 ).sort_values(by='date',ascending=False)
df_wide2 = df_all_prob.pivot_table(index='date', columns=['user'], values=['probability'], aggfunc='sum',fill_value=0 ).sort_values(by='date',ascending=False).droplevel(0, axis=1) 
df_wide_merge = pd.merge(df_wide1, df_wide2, how='inner', on='date')

- Merging Sat/Sun Tweets to Monday and re-merging to data

In [22]:
# Drop Saturday-Monday And replace with Monday
week_end_mask = df_wide_merge.reset_index().date.dt.day_name().isin(['Saturday', 'Sunday', 'Monday'])
week_end = df_wide_merge.reset_index().loc[week_end_mask, :]
monday_group = week_end.groupby([pd.Grouper(key='date', freq='W-MON')])[df_wide_merge.columns].sum().reset_index('date')

df_wide_stripped = df_wide_merge.reset_index().loc[~ week_end_mask, :]
df_wide = pd.merge(df_wide_stripped, monday_group, how='outer').set_index('date')
df_wide.head(5)

,favorite_count,retweet_count,AswathDamodaran_twitter,BillGates_twitter,BobPisani_twitter,DanZanger_twitter,DipFinding_twitter,EconguyRosie_twitter,GRDecter_twitter,HindenburgRes_twitter,JeffBezos_twitter,JehoshaphatRsch_twitter,KennethLFisher_twitter,MITickWatcher_twitter,MacroCharts_twitter,MarketMagnifier_twitter,OptionsProVol_twitter,RayDalio_twitter,ResearchGrizzly_twitter,TwentyonTwenty__twitter,WatcherGuru_twitter,andrewrsorkin_twitter,biancoresearch_twitter,bsc_daily_twitter,cstewartcfa_twitter,eWhispers_twitter,jimcramer_twitter,lei_zhang_lz_twitter,markminervini_twitter,muddywatersre_twitter,prrobbins_twitter,realwillmeade_twitter,script_crypto_twitter,techbudsolution_twitter
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-02-03,35508,5629,0.019219,0.004562,0.000260,0.0,0.126416,0.004234,0.006970,0.0,0.000000,0.0,0.052380,0.153458,0.000000,0.0,0.017344,0.002671,0.0,0.012475,0.006944,0.000000,0.002646,0.051548,0.000705,0.007442,0.008453,0.000000,0.002395,0.005878,0.000313,0.000000,0.003701,0.000034
2023-02-02,55531,7141,0.021234,0.001204,0.015224,0.0,0.058315,0.007141,0.007865,0.0,0.000000,0.0,0.049809,0.206806,0.002318,0.0,0.014951,0.013856,0.0,0.034361,0.004568,0.000000,0.000720,0.048649,0.000000,0.016562,0.000614,0.000025,0.004613,0.000000,0.006279,0.025660,0.005311,0.000000
2023-02-01,91960,13980,0.000000,0.000000,0.018260,0.0,0.140774,0.012067,0.010200,0.0,0.000000,0.0,0.036095,0.230284,0.000000,0.0,0.015966,0.007884,0.0,0.024758,0.004377,0.002536,0.019865,0.013837,0.004732,0.014864,0.015832,0.003604,0.012069,0.000000,0.002181,0.015748,0.005103,0.002132
2023-01-31,117675,17073,0.000000,0.001423,0.004139,0.0,0.090134,0.000000,0.043312,0.0,0.004599,0.0,0.015100,0.209838,0.000000,0.0,0.014368,0.000296,0.0,0.032020,0.005684,0.000000,0.003263,0.032768,0.000074,0.012324,0.005221,0.000184,0.001081,0.000000,0.000141,0.039290,0.004597,0.005290
2023-01-27,105208,18175,0.000410,0.000045,0.001317,0.0,0.096108,0.000000,0.033335,0.0,0.000000,0.0,0.034561,0.163753,0.000000,0.0,0.015305,0.001451,0.0,0.031353,0.006239,0.000000,0.005478,0.023758,0.000000,0.010284,0.006765,0.006623,0.000000,0.002167,0.002397,0.000000,0.001897,0.006292


In [23]:
path_all_merged_twitter_analysts_pivot = f'../Sentiment_Analysis/Stock_Market/data/merge/all_merged_twitter_users' # Create Folders
if not os.path.exists(path_all_merged_twitter_analysts_pivot):
    os.makedirs(path_all_merged_twitter_analysts_pivot)
df_wide.to_csv(path_all_merged_twitter_analysts_pivot +'/all_merged_twitter_users_pivot.csv', index=True) # Export to csv

df_wide.head(5)

,favorite_count,retweet_count,AswathDamodaran_twitter,BillGates_twitter,BobPisani_twitter,DanZanger_twitter,DipFinding_twitter,EconguyRosie_twitter,GRDecter_twitter,HindenburgRes_twitter,JeffBezos_twitter,JehoshaphatRsch_twitter,KennethLFisher_twitter,MITickWatcher_twitter,MacroCharts_twitter,MarketMagnifier_twitter,OptionsProVol_twitter,RayDalio_twitter,ResearchGrizzly_twitter,TwentyonTwenty__twitter,WatcherGuru_twitter,andrewrsorkin_twitter,biancoresearch_twitter,bsc_daily_twitter,cstewartcfa_twitter,eWhispers_twitter,jimcramer_twitter,lei_zhang_lz_twitter,markminervini_twitter,muddywatersre_twitter,prrobbins_twitter,realwillmeade_twitter,script_crypto_twitter,techbudsolution_twitter
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-02-03,35508,5629,0.019219,0.004562,0.000260,0.0,0.126416,0.004234,0.006970,0.0,0.000000,0.0,0.052380,0.153458,0.000000,0.0,0.017344,0.002671,0.0,0.012475,0.006944,0.000000,0.002646,0.051548,0.000705,0.007442,0.008453,0.000000,0.002395,0.005878,0.000313,0.000000,0.003701,0.000034
2023-02-02,55531,7141,0.021234,0.001204,0.015224,0.0,0.058315,0.007141,0.007865,0.0,0.000000,0.0,0.049809,0.206806,0.002318,0.0,0.014951,0.013856,0.0,0.034361,0.004568,0.000000,0.000720,0.048649,0.000000,0.016562,0.000614,0.000025,0.004613,0.000000,0.006279,0.025660,0.005311,0.000000
2023-02-01,91960,13980,0.000000,0.000000,0.018260,0.0,0.140774,0.012067,0.010200,0.0,0.000000,0.0,0.036095,0.230284,0.000000,0.0,0.015966,0.007884,0.0,0.024758,0.004377,0.002536,0.019865,0.013837,0.004732,0.014864,0.015832,0.003604,0.012069,0.000000,0.002181,0.015748,0.005103,0.002132
2023-01-31,117675,17073,0.000000,0.001423,0.004139,0.0,0.090134,0.000000,0.043312,0.0,0.004599,0.0,0.015100,0.209838,0.000000,0.0,0.014368,0.000296,0.0,0.032020,0.005684,0.000000,0.003263,0.032768,0.000074,0.012324,0.005221,0.000184,0.001081,0.000000,0.000141,0.039290,0.004597,0.005290
2023-01-27,105208,18175,0.000410,0.000045,0.001317,0.0,0.096108,0.000000,0.033335,0.0,0.000000,0.0,0.034561,0.163753,0.000000,0.0,0.015305,0.001451,0.0,0.031353,0.006239,0.000000,0.005478,0.023758,0.000000,0.010284,0.006765,0.006623,0.000000,0.002167,0.002397,0.000000,0.001897,0.006292


### CHECKPOINT    
    Load pivot data

In [24]:
path_all_merged_twitter_analysts_pivot = f'../Sentiment_Analysis/Stock_Market/data/merge/all_merged_twitter_users'
df_wide = pd.read_csv(path_all_merged_twitter_analysts_pivot +'/all_merged_twitter_users_pivot.csv').astype({'date':'datetime64[ns]'}).set_index('date')
df_wide.head()

,favorite_count,retweet_count,AswathDamodaran_twitter,BillGates_twitter,BobPisani_twitter,DanZanger_twitter,DipFinding_twitter,EconguyRosie_twitter,GRDecter_twitter,HindenburgRes_twitter,JeffBezos_twitter,JehoshaphatRsch_twitter,KennethLFisher_twitter,MITickWatcher_twitter,MacroCharts_twitter,MarketMagnifier_twitter,OptionsProVol_twitter,RayDalio_twitter,ResearchGrizzly_twitter,TwentyonTwenty__twitter,WatcherGuru_twitter,andrewrsorkin_twitter,biancoresearch_twitter,bsc_daily_twitter,cstewartcfa_twitter,eWhispers_twitter,jimcramer_twitter,lei_zhang_lz_twitter,markminervini_twitter,muddywatersre_twitter,prrobbins_twitter,realwillmeade_twitter,script_crypto_twitter,techbudsolution_twitter
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-02-03,35508,5629,0.019219,0.004562,0.000260,0.0,0.126416,0.004234,0.006970,0.0,0.000000,0.0,0.052380,0.153458,0.000000,0.0,0.017344,0.002671,0.0,0.012475,0.006944,0.000000,0.002646,0.051548,0.000705,0.007442,0.008453,0.000000,0.002395,0.005878,0.000313,0.000000,0.003701,0.000034
2023-02-02,55531,7141,0.021234,0.001204,0.015224,0.0,0.058315,0.007141,0.007865,0.0,0.000000,0.0,0.049809,0.206806,0.002318,0.0,0.014951,0.013856,0.0,0.034361,0.004568,0.000000,0.000720,0.048649,0.000000,0.016562,0.000614,0.000025,0.004613,0.000000,0.006279,0.025660,0.005311,0.000000
2023-02-01,91960,13980,0.000000,0.000000,0.018260,0.0,0.140774,0.012067,0.010200,0.0,0.000000,0.0,0.036095,0.230284,0.000000,0.0,0.015966,0.007884,0.0,0.024758,0.004377,0.002536,0.019865,0.013837,0.004732,0.014864,0.015832,0.003604,0.012069,0.000000,0.002181,0.015748,0.005103,0.002132
2023-01-31,117675,17073,0.000000,0.001423,0.004139,0.0,0.090134,0.000000,0.043312,0.0,0.004599,0.0,0.015100,0.209838,0.000000,0.0,0.014368,0.000296,0.0,0.032020,0.005684,0.000000,0.003263,0.032768,0.000074,0.012324,0.005221,0.000184,0.001081,0.000000,0.000141,0.039290,0.004597,0.005290
2023-01-27,105208,18175,0.000410,0.000045,0.001317,0.0,0.096108,0.000000,0.033335,0.0,0.000000,0.0,0.034561,0.163753,0.000000,0.0,0.015305,0.001451,0.0,0.031353,0.006239,0.000000,0.005478,0.023758,0.000000,0.010284,0.006765,0.006623,0.000000,0.002167,0.002397,0.000000,0.001897,0.006292


In [25]:
with open(os.path.normpath(os.getcwd() + '/Stock_Market/ticker_list/ticker_list.xlsx'), 'rb') as f:
    ticker_df = pd.read_excel(f, sheet_name='ticker_sheet')
    ticker_df = ticker_df.where(pd.notnull(ticker_df), '')
    f.close()
ticker_df.head(10)

,ticker_name,ticker_label
0,^GSPC,SandP_500
1,^IXIC,NASDAQ
2,^RUT,RUSSEL
3,^DJI,DOW_JONES
4,AAPL,APPLE
5,ABBV,ABBVIE
6,ABNB,AIRBNB
7,ADBE,ADOBE
8,AMD,AMD
9,AMZN,AMAZON


In [26]:
# downloding index fund's or stock tickers  #.resample('D').ffill()
how_far_back = df_wide.index.min().date()
today = date.today()
column_names = dict(zip(ticker_df.ticker_name, ticker_df.ticker_label))
column_names['Date']='date'
stock_list = list(ticker_df.ticker_name)
stock_str = ' '.join( stock_list )

index_funds_df = yf.download(stock_str, how_far_back, today, interval = '1d', progress=False)['Close'].reset_index('Date').rename(columns=column_names)

convert_dict = dict(zip(ticker_df.ticker_label, ['float64']*len(ticker_df.ticker_label)))
convert_dict['date'] = 'datetime64[ns]'
index_funds_df = index_funds_df.astype(convert_dict)

print(f'{how_far_back} -> {today}')

2017-01-03 -> 2023-02-03


In [27]:
path_index_funds_merge = f'../Sentiment_Analysis/Stock_Market/data/merge/all_merged_index_funds' # Create Folders
if not os.path.exists(path_index_funds_merge):
    os.makedirs(path_index_funds_merge)
index_funds_df.to_csv(path_index_funds_merge +'/all_merged_index_funds.csv', index=False) # Export to csv
index_funds_df.head(5)

,date,APPLE,ABBVIE,AIRBNB,ADOBE,AMD,AMAZON,ARK_INNOVATION,ASML_Holding,BROADCOM,BOEING,ALIBABA,BandG_Foods,Biogen,CATERPILLAR,COSTCO,SALESFORCE,CROWDSTRIKE,CISCO,CHEVRON,DANAHER,DISNEY,DEVON_ENERGY,EBAY,ESTEE_LAUDER,ETSY,GENERAL_ELECTRIC,GOOGLE,HALLIBURTON,HONEYWELL,IBM,JNJ,JPMORGAN,KINDER_MORGAN,ELI_LILLY,LEMONADE,MASTERCARD,MICROCHIP,META,MARVELL,MORGAN_STANLEY,MICROSOFT,NETFLIX,NIKE,Service_Now,NUCOR,NVIDIA,NVE,REALTY_INCOME,OKTA,ORACLE,PALO_ALTO,UIPATH,PROCTER_GAMBLE,PAYPAL,QUALCOMM,ROBLOX,STARBUCKS,SHOPIFY,SNOWFLAKE,SPLUNK,SQUARE_BLOCK,CONSTELLATION_BRANDS,SKYWORKS,TELADOC,ATLASSIAN,TESLA,TAIWAN_SEMICONDUCTOR,VISA,VERIZON,WALMART,DOW_JONES,SandP_500,NASDAQ,RUSSEL
0,2017-01-03,29.037500,62.410000,NaN,103.480003,11.43,37.683498,20.437000,110.449997,178.339996,156.970001,88.599998,43.250000,291.709991,93.989998,159.729996,70.540001,NaN,30.540001,117.849998,78.779999,106.080002,47.080002,29.840000,77.330002,11.88,190.296036,39.306999,55.680000,111.352638,159.837479,115.839996,87.230003,21.610001,74.599998,NaN,105.389999,31.924999,116.860001,14.05,43.049999,62.580002,127.489998,51.980000,75.660004,59.610001,25.502501,70.430000,55.707363,NaN,38.549999,42.313332,NaN,84.199997,40.250000,65.400002,NaN,55.349998,4.282,NaN,52.500000,13.81,154.750000,75.150002,16.500000,24.600000,14.466000,29.309999,79.500000,54.580002,68.660004,19881.759766,2257.830078,5429.080078,1365.489990
1,2017-01-04,29.004999,63.290001,NaN,104.139999,11.43,37.859001,21.360001,109.919998,177.070007,158.619995,90.510002,43.900002,294.049988,93.570000,159.759995,72.800003,NaN,30.100000,117.820000,79.279999,107.440002,47.500000,29.760000,78.699997,11.97,190.356094,39.345001,55.889999,111.659348,161.816437,115.650002,86.910004,21.719999,74.720001,NaN,106.389999,31.910000,118.690002,14.24,43.619999,62.299999,129.410004,53.070000,78.830002,61.250000,26.097500,71.470001,56.540699,NaN,38.740002,42.666668,NaN,84.500000,41.000000,65.470001,NaN,55.990002,4.414,NaN,54.840000,14.25,157.990005,74.989998,16.500000,25.150000,15.132667,29.299999,80.150002,54.520000,69.059998,19942.160156,2270.750000,5477.000000,1387.949951
2,2017-01-05,29.152500,63.770000,NaN,105.910004,11.24,39.022499,21.150000,111.239998,174.279999,158.710007,94.370003,44.000000,293.570007,93.000000,162.910004,72.790001,NaN,30.170000,117.309998,80.010002,107.379997,49.009998,30.010000,78.599998,12.40,189.275208,39.701000,56.209999,111.908539,161.281067,116.860001,86.110001,21.690001,75.589996,NaN,106.989998,31.295000,120.669998,14.04,43.220001,62.299999,131.809998,53.060001,79.129997,60.900002,25.434999,71.360001,58.062016,NaN,38.639999,43.990002,NaN,85.059998,41.060001,65.550003,NaN,56.459999,4.768,NaN,54.560001,14.56,146.750000,74.570000,16.400000,24.969999,15.116667,29.799999,81.089996,54.639999,69.209999,19899.289062,2269.000000,5487.939941,1371.939941
3,2017-01-06,29.477501,63.790001,NaN,108.300003,11.32,39.799500,21.365000,111.120003,176.589996,159.100006,93.889999,43.500000,295.000000,93.040001,162.830002,73.800003,NaN,30.230000,116.839996,80.430000,108.980003,48.669998,31.049999,79.160004,13.08,189.815643,40.307499,56.660000,113.604996,162.074570,116.300003,86.120003,21.809999,75.669998,NaN,107.760002,31.840000,123.410004,14.18,43.849998,62.840000,131.070007,53.910000,82.099998,60.290001,25.775000,71.730003,57.984497,NaN,38.450001,45.016666,NaN,85.029999,41.450001,65.529999,NaN,57.130001,4.690,NaN,56.169998,15.00,149.440002,74.959999,16.549999,24.719999,15.267333,29.629999,82.209999,53.259998,68.260002,19963.800781,2276.979980,5521.060059,1367.280029
4,2017-01-09,29.747499,64.209999,NaN,108.570000,11.49,39.846001,21.645000,112.489998,176.970001,158.320007,94.720001,43.299999,299.019989,92.370003,160.970001,73.959999,NaN,30.180000,115.839996,80.589996,108.360001,46.580002,30.750000,78.239998,12.35,188.914917,40.332500,56.070000,112.665718,160.277252,116.279999,86.180000,21.639999,76.269997,NaN,107.550003,32.209999,124.900002,14.37,42.709999,62.639999,130.949997,53.380001,82.000000,59.639999,26.820000,7

In [28]:
# Merging the probabilities of words used from twitter and database of index funds
df_merge = pd.merge(index_funds_df, df_wide, how='inner', on='date').set_index('date').fillna(0)
df_merge_original = df_merge.copy()

columns = list(ticker_df.ticker_label) + ['favorite_count', 'retweet_count']
df_merge = normalize_columns(df_merge.copy(), columns)
df_merge.tail(5)

,APPLE,ABBVIE,AIRBNB,ADOBE,AMD,AMAZON,ARK_INNOVATION,ASML_Holding,BROADCOM,BOEING,ALIBABA,BandG_Foods,Biogen,CATERPILLAR,COSTCO,SALESFORCE,CROWDSTRIKE,CISCO,CHEVRON,DANAHER,DISNEY,DEVON_ENERGY,EBAY,ESTEE_LAUDER,ETSY,GENERAL_ELECTRIC,GOOGLE,HALLIBURTON,HONEYWELL,IBM,JNJ,JPMORGAN,KINDER_MORGAN,ELI_LILLY,LEMONADE,MASTERCARD,MICROCHIP,META,MARVELL,MORGAN_STANLEY,MICROSOFT,NETFLIX,NIKE,Service_Now,NUCOR,NVIDIA,NVE,REALTY_INCOME,OKTA,ORACLE,PALO_ALTO,UIPATH,PROCTER_GAMBLE,PAYPAL,QUALCOMM,ROBLOX,STARBUCKS,SHOPIFY,SNOWFLAKE,SPLUNK,SQUARE_BLOCK,CONSTELLATION_BRANDS,SKYWORKS,TELADOC,ATLASSIAN,TESLA,TAIWAN_SEMICONDUCTOR,VISA,VERIZON,WALMART,DOW_JONES,SandP_500,NASDAQ,RUSSEL,favorite_count,retweet_count,AswathDamodaran_twitter,BillGates_twitter,BobPisani_twitter,DanZanger_twitter,DipFinding_twitter,EconguyRosie_twitter,GRDecter_twitter,HindenburgRes_twitter,JeffBezos_twitter,JehoshaphatRsch_twitter,KennethLFisher_twitter,MITickWatcher_twitter,MacroCharts_twitter,MarketMagnifier_twitter,OptionsProVol_twitter,RayDalio_twitter,ResearchGrizzly_twitter,TwentyonTwenty__twitter,WatcherGuru_twitter,andrewrsorkin_twitter,biancoresearch_twitter,bsc_daily_twitter,cstewartcfa_twitter,eWhispers_twitter,jimcramer_twitter,lei_zhang_lz_twitter,markminervini_twitter,muddywatersre_twitter,prrobbins_twitter,realwillmeade_twitter,script_crypto_twitter,techbudsolution_twitter
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-01-27,0.764191,0.750522,0.534680,0.456889,0.432275,0.433594,0.146559,0.715246,0.835528,0.336101,0.217449,0.080658,0.451380,1.000000,0.771071,0.392532,0.355925,0.545321,0.935739,0.738025,0.215475,0.824177,0.419835,0.656843,0.446285,0.313604,0.551184,0.669963,0.800108,0.525372,0.762522,0.660809,0.669753,0.891333,0.082669,0.922125,0.810136,0.214239,0.390665,0.853312,0.661871,0.413470,0.605463,0.612072,0.933832,0.580138,0.303727,0.700691,0.244636,0.775153,0.712726,0.179511,0.746542,0.157938,0.600515,0.280211,0.780186,0.277938,0.396526,0.250044,0.261269,0.792420,0.363034,0.046991,0.322123,0.416966,0.574751,0.886309,0.197979,0.824116,0.845034,0.716313,0.582651,0.633999,0.210350,0.211359,0.000410,0.000045,0.001317,0.000000,0.096108,0.000000,0.033335,0.000000,0.000000,0.0,0.034561,0.163753,0.000000,0.0,0.015305,0.001451,0.0,0.031353,0.006239,0.000000,0.005478,0.023758,0.000000,0.010284,0.006765,0.006623,0.000000,0.002167,0.002397,0.000000,0.001897,0.006292
2023-01-30,0.745041,0.745042,0.504888,0.444425,0.412915,0.422243,0.132750,0.693332,0.816690,0.331761,0.189181,0.083128,0.433156,0.982443,0.771050,0.393493,0.345760,0.536786,0.896510,0.725291,0.198913,0.783836,0.417815,0.658269,0.435497,0.298226,0.526409,0.666604,0.782075,0.536290,0.679311,0.647979,0.621914,0.886009,0.083488,0.912151,0.777597,0.198281,0.361527,0.842190,0.642463,0.399894,0.596306,0.586053,0.930460,0.541317,0.320429,0.686996,0.237199,0.750000,0.700346,0.171170,0.751260,0.149803,0.587861,0.263732,0.772188,0.260399,0.380502,0.240517,0.246194,0.776784,0.344465,0.041166,0.305261,0.388728,0.573224,0.872659,0.213323,0.811910,0.830700,0.695685,0.561209,0.616266,0.469726,0.725832,0.021699,0.000000,0.014598,0.000036,0.058394,0.000000,0.031528,0.004337,0.000337,0.0,0.017300,0.245524,0.000000,0.0,0.017321,0.004932,0.0,0.088529,0.006435,0.000004,0.005687,0.100092,0.002815,0.015471,0.017896,0.007264,0.002086,0.018651,0.004369,0.066191,0.003911,0.002271
2023-01-31,0.753472,0.763309,0.512405,0.456257,0.430634,0.439572,0.143180,0.706842,0.823720,0.341396,0.185243,0.080110,0.454990,0.929252,0.788226,0.406942,0.361212,0.550029,0.895166,0.731717,0.206557,0.796227,0.429017,0.678199,0.445519,0.295984,0.543644,0.683022,0.802793,0.529451,0.698277,0.656927,0.641975,0.898153,0.088836,0.910368,0.809160,0.204794,0.376483,0.859120,0.660625,0.401223,0.603884,0.606114,0.948334,0.553419,0.327619,0.688547,0.252279,0.767025,0.706161,0.180451,0.765948,0.156675,0.599156,0.276202,0.781734,0.273022,0.389261,0.252908,0.253396,0.809987,0.352310,0.04673

In [29]:
path_twitter_and_index_fund = f'../Sentiment_Analysis/Stock_Market/data/merge/combined'
if not os.path.exists(path_twitter_and_index_fund):
    os.makedirs(path_twitter_and_index_fund)
df_merge.to_csv(path_twitter_and_index_fund +'/index_funds_and_twitter_analysts.csv') # Export to csv

In [30]:
path_twitter_and_index_fund = f'../Sentiment_Analysis/Stock_Market/data/merge/combined'
df_merge = pd.read_csv(path_twitter_and_index_fund +'/index_funds_and_twitter_analysts.csv').set_index('date')

# Prediction of Today 

- trained for 5 day averages ( 0 for a bad day and 1  for a good day )

In [31]:
# Todays Data
todays_test = download_todays_test(ticker_df, df_wide, df_merge_original)
Xnew = sm.add_constant(todays_test, has_constant='add')

model = {} # Model Build For Each index fund
print(f"date: { todays_test.index.date.max() }")
output = pd.DataFrame(columns=['index', 'prediction'])
for t in ticker_df.ticker_label:
    data_with_target = create_target(df_merge.copy(), day = 5, ticker = t)
    m = linear_model(data_with_target,split=0.20,summary = False)
    y_pred = m['lm'].predict(Xnew)
    model[t] = (y_pred, m)
    output = pd.concat([output, pd.DataFrame.from_records([(t, y_pred[0])], columns=['index', 'prediction'])])
    
pd.set_option('display.max_rows', 500)
display(output.sort_values(by=['prediction'], ascending=False))

date: 2023-02-03


,index,prediction
0,NUCOR,1.190847
0,VISA,1.006677
0,SPLUNK,1.004027
0,EBAY,0.999825
0,APPLE,0.992455
0,LEMONADE,0.978971
0,RUSSEL,0.970240
0,AMD,0.967545
0,NVIDIA,0.953408
0,BROADCOM,0.947712
